# Accessing the project directory on my Google Drive  

In [1]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)
project_dir = "/drive/My Drive/RNNSeq2Seq/"

# change working directory to project_dir
os.chdir(project_dir)

Mounted at /drive


# Dependencies

In [2]:
from scripts.model import *
from scripts.dataloader import create_dataloader
from scripts.utils import *
from scripts.pytorch_utils import *
import matplotlib.pyplot as plt

# Helper function to help visualize 

In [3]:
def plot_log(log):
    len_range = len(log) -1 if "Best eval accu" in log else len(log)
    epoch_nums, train_loss, dev_loss = [], [], []
    train_acc, dev_acc = [], []

    for epoch in range(1, len_range+1):
        epoch_nums.append(epoch)
        train = log[f"Epoch#{epoch}"]["Train"]
        dev = log[f"Epoch#{epoch}"]["Eval"]
        train_loss.append(train['loss'])
        train_acc.append(train['abosulate accuracy'])

        dev_loss.append(dev['loss'])
        dev_acc.append(dev['abosulate accuracy'])

    ax1 = plt.subplot(211)
    ax1.plot(epoch_nums, train_loss, label="train loss")
    ax1.plot(epoch_nums, dev_loss, label="dev loss")
    ax1.legend()

    ax2 = plt.subplot(212)
    ax2.plot(epoch_nums, train_acc, label="train acc")
    ax2.plot(epoch_nums, dev_acc, label="dev acc")
    ax2.legend()
    plt.show()

In [4]:
folder = "tuning_data/"
experiment_num = 4
shuffle_batch = True

# must be the num of seq per seq length if shuffle_batch is False
# this "tr_batch_size" will be used for dev, and shuffled test sets
tr_batch_size = 250 
dev_batch_size = 500 if not shuffle_batch else tr_batch_size

In [5]:
# setups for training and evaluting

print_freq = 5
acc_threshold = 0.5 # for saving the best trained models 
max_epoch_num = 150 # max num of epoches 
train_acc_exit=0.9999 # train acc exit threshold (eval_acc_exit must also be met)
eval_acc_exit=0.995 # dev acc exit threshold (train_acc_exit must also be met)
teacher_forcing_ratio = 1.0 # probability of using real symbol from target sequence for training

# RevStr

In [6]:
train = read_data(join(folder, "RevStr/train.txt"))
dev = read_data(join(folder, "RevStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_lstm_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1

In [7]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_vocab_size = len(in_vocab) + 2
out_vocab_size = len(out_vocab) + 2

hidden_size = 256
embd_dim = 128
num_layers = 1
rnn_type = "LSTM"
dropout_rate = 0.0
bidirectional = False
use_attention = True
reduction_method = torch.sum

learning_rate = 5e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 1.9733384996652603, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.000691666679631453, 'overlap rate': 0.4177700437605381}
evaluation performance: {'loss': 3.672946700453758, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007066666810715105, 'overlap rate': 0.4251525416970253}

Current epoch: 10, 
training performance: {'loss': 1.973444402217865, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007216666817839723, 'overlap rate': 0.4177967131137848}
evaluation performance: {'loss': 3.568504822254181, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007141666839743266, 'overlap rate': 0.30127086453139784}

Current epoch: 15, 
training performance: {'loss': 1.9548727989196777, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007166666779085063, 'overlap rate': 0.42727004587650297}
evaluation performance: {'loss': 3.261419615149498, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.000

KeyboardInterrupt: ignored

## Hyperparameter set 2


In [ ]:
# hidden_size = 128
# embd_dim = 128
# num_layers = 1
# rnn_type = "LSTM"
# encoder = Encoder(in_vocab_size, hidden_size, 
#                   embd_dim, num_layers, rnn_type, 
#                   dropout_rate, bidirectional, 
#                   reduction_method)
# attention = Attention(hidden_size)
# decoder = Decoder(out_vocab_size, hidden_size, 
#                   embd_dim, num_layers, rnn_type,
#                   attention, use_attention, 
#                   dropout_rate)

# model = Seq2Seq(encoder, decoder, device).to(device)
# model.apply(init_weights)

# learning_rate = 5e-3
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
#                          saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
#                          train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

# RedStr



In [9]:
train = read_data(join(folder, "RedStr/train.txt"))
dev = read_data(join(folder, "RedStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1


In [10]:
hidden_size = 256
embd_dim = 128
num_layers = 1
rnn_type = "LSTM"
encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

learning_rate = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 1.9788112342357635, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.000671666681591887, 'overlap rate': 0.41764337196946144}
evaluation performance: {'loss': 3.0700750648975372, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0006641666791438183, 'overlap rate': 0.3486650358885527}

Current epoch: 10, 
training performance: {'loss': 1.9513654857873917, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007683333511522506, 'overlap rate': 0.428023374825716}
evaluation performance: {'loss': 2.183637002110481, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0006216666823092964, 'overlap rate': 0.41111087389290335}

Current epoch: 15, 
training performance: {'loss': 1.9753008425235747, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0007600000149977859, 'overlap rate': 0.4237700439989567}
evaluation performance: {'loss': 3.588314163684845, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.00

KeyboardInterrupt: ignored

In [11]:
plot_log(log)